In [18]:
import numpy as np
import random
from qpsolvers import solve_qp
import matplotlib.pyplot as plt

In [21]:
# Replace with actual digits from registration number
dig1 = 0
dig2 = 0
dummyrepetitions = 10 * dig1 + dig2

# Parameters
n = 10
random.seed(42)

# Generate data
for _ in range(dummyrepetitions):
    dummy = random.uniform(0, 1)

Corr = np.array([[0] * n for _ in range(n)], dtype=float)
for i in range(n):
    for j in range(n):
        Corr[i][j] = (-1) ** abs(i - j) / (abs(i - j) + 1)

ssigma = np.zeros((n, 1), dtype=float)
mmu = np.zeros((n, 1), dtype=float)

ssigma[0] = 2
mmu[0] = 3

for i in range(n - 1):
    ssigma[i + 1] = ssigma[i] + 2 * random.uniform(0, 1)
    mmu[i + 1] = mmu[i] + 1

ddiag = np.zeros((n, n), dtype=float)
np.fill_diagonal(ddiag, ssigma.flatten())
C2 = np.matmul(np.matmul(ddiag, Corr), ddiag)

# Ensure C is positive semi-definite
C = 0.5 * (C2 + C2.T)
eigvals = np.linalg.eigvals(C)
if np.any(eigvals < 0):
    C -= np.min(eigvals) * np.eye(n)

# μ is the expected returns vector
mu = mmu.flatten()

In [22]:
def solve_task_1(r_values):
    sigmas = []
    mus = []

    for r in r_values:
        # Quadratic programming setup
        P = C
        q = np.zeros(n)
        G = -np.eye(n)
        h = np.zeros(n)
        A = np.vstack([mu, np.ones(n)])
        b = np.array([r, 1])

        # Print debug information
        print(f"Solving for r = {r}")
        print("P:", P)
        print("q:", q)
        print("G:", G)
        print("h:", h)
        print("A:", A)
        print("b:", b)

        # Solve QP problem
        x = solve_qp(P, q, G, h, A, b, solver='quadprog')
        if x is None:
            print(f"No solution found for r = {r}")
            continue

        sigma = np.sqrt(np.dot(x.T, np.dot(C, x)))
        mu_val = np.dot(mu.T, x)

        sigmas.append(sigma)
        mus.append(mu_val)

    return sigmas, mus

# Define r values
r_values = np.arange(2.00, 9.25, 0.25)

# Solve task 1
sigmas, mus = solve_task_1(r_values)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(sigmas, mus, marker='o')
plt.xlabel('σ (Risk)')
plt.ylabel('μ (Return)')
plt.title('Efficient Frontier')
plt.grid(True)
plt.show()

Solving for r = 2.0
P: [[  4.          -3.2788536    2.21925007  -1.93946687   1.73014209
   -1.93276588   2.04334189  -2.23401394   2.02442965  -1.99075541]
 [ -3.2788536   10.75088091  -5.45744706   4.23948529  -3.54555163
    3.80235382  -3.9082277    4.18571694  -3.73376725   3.62633086]
 [  2.21925007  -5.45744706  11.08140948  -6.45624299   4.79952244
   -4.82545218   4.76142097  -4.9578356    4.33226152  -4.14186009]
 [ -1.93946687   4.23948529  -6.45624299  15.04612699  -8.38888316
    7.4970708   -6.93523934   6.93247364  -5.88947136   5.51572025]
 [  1.73014209  -3.54555163   4.79952244  -8.38888316  18.70869778
  -12.5398485   10.31120944  -9.66290386   7.88073962  -7.1756036 ]
 [ -1.93276588   3.80235382  -4.82545218   7.4970708  -12.5398485
   33.62025562 -20.73383286  17.2713037  -13.20552639  11.54299243]
 [  2.04334189  -3.9082277    4.76142097  -6.93523934  10.31120944
  -20.73383286  51.1467646  -31.9539799   21.71716004 -17.79659845]
 [ -2.23401394   4.18571694  -4.9

SolverNotFound: found solvers [] but 'quadprog' is not one of them; if 'quadprog' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can run ``pip install qpsolvers[quadprog]``